# Tugas 3 Pengenalan Pola 
# Diagnosa Penyakit
## Nadya Avirianta S
### 16/394096/PA/17187

### Data Preparation

In [0]:
#import library yang dibutuhkan 
import pandas as pd
import numpy as np
import nltk
import math
#nltk.download('stopwords')

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data.csv to data.csv
User uploaded file "data.csv" with length 10610 bytes


In [0]:
data = pd.read_csv("data.csv", )

In [0]:
data.head()

,No,Gejala,Diagnosis
0,1,"menggigil, demam, sakit kepala",Malaria (bentuk benigma)
1,2,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid
2,3,"Mata lengket, mata berair, pandangan sedikit k...",
3,4,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis
4,5,"Gusi bengkak, gusi kemerahan, gusi berdarah",


In [0]:
data.dtypes

No            int64
Gejala       object
Diagnosis    object
dtype: object

In [0]:
data = data.fillna(' ')

### Preprocessing

In [0]:
#Casefolding & Tokenizing & Filtering

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
stop_words = set(stopwords.words('indonesian')) 



def preprocess(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8").replace(u"\ufffd", "?")
    except:
        clean = stripped

    #menghapus simbol2 
    letters_only = re.sub("[^a-zA-Z0-9]", " ", clean)
    #casefolding
    lower_case = letters_only.lower()
    #tokenization
    words = tok.tokenize(lower_case)
    #untuk filter dengan menghilangkan stopwords 
    filtered_words = [w for w in words if not w in stop_words] 
    return (" ".join(filtered_words)).strip()
    
    #return ([filtered_words])

In [0]:
print ("Cleaning...\n")
cleandata = []
for i in range(len(data)):
    cleandata.append(preprocess(data['Gejala'][i]))
print ("Done")

Cleaning...

Done


In [0]:
print ("Cleaning...\n")
cleandia = []
for i in range(len(data)):
    cleandia.append(preprocess(data['Diagnosis'][i]))
print ("Done")

Cleaning...

Done


In [0]:
cleandf = pd.DataFrame(cleandata,columns=['gejala'])
cleandf['diagnosis'] = cleandia
cleandf.head()

,gejala,diagnosis
0,menggigil demam sakit kepala,malaria bentuk benigma
1,kaku kuduk penurunan kesadaran muntah proyekti...,meningitis perdarahan subarachnoid
2,mata lengket mata berair pandangan kabur kelop...,
3,pipi bengkak nyeri mengunyah nyeri testis,parotitis
4,gusi bengkak gusi kemerahan gusi berdarah,


In [0]:
cleandf.to_csv('cleandata.csv',index=False , encoding='utf-8')

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving cleandata.csv to cleandata.csv
User uploaded file "cleandata.csv" with length 8324 bytes


In [0]:
cleandata = pd.read_csv('cleandata.csv')

In [4]:
cleandata.head()

,gejala,diagnosis
0,menggigil demam sakit kepala,malaria bentuk benigma
1,kaku kuduk penurunan kesadaran muntah proyekti...,meningitis perdarahan subarachnoid
2,mata lengket mata berair pandangan kabur kelop...,NaN
3,pipi bengkak nyeri mengunyah nyeri testis,parotitis
4,gusi bengkak gusi kemerahan gusi berdarah,NaN


### Feature Extraction & Representation dengan TF/IDF

In [0]:
cleandata.isnull().sum()

gejala       0
diagnosis    8
dtype: int64

In [0]:
#membuat list gejala yang belum ada diagnosisnya 
test = cleandata[cleandata.isnull().any(axis=1)]

In [0]:
test

,gejala,diagnosis
2,mata lengket mata berair pandangan kabur kelop...,NaN
4,gusi bengkak gusi kemerahan gusi berdarah,NaN
18,batuk minggu sesak napas nyeri dada dahak berd...,NaN
19,demam menggigil suhu tubuh meningkat batuk ber...,NaN
29,demam muntah diare cair ampas biji lombok fese...,NaN
36,nyeri kolik daerah pinggang malaise mual kenci...,NaN
67,suhu tubuh batuk disertai sputum purulent berw...,NaN
76,ruam gatal terdri macula makula lesi demam tuj...,NaN


In [0]:
#Hitung frekuensi kemunculan term (TF) di tiap gejala

def TF(text):
  dictTF = {}
  for word in text:
    if word in dictTF:
      dictTF[word] += 1
    else:
      dictTF[word] = 1
  for word in dictTF:
    dictTF[word] = dictTF[word] / len(text)
  return dictTF

In [0]:
TFdict = []
for i in range(len(cleandata)):
  TFdict.append(TF(cleandata['gejala'][i].split()))

In [8]:
#contoh TFdict
TFdict[3]

{'bengkak': 0.16666666666666666,
 'mengunyah': 0.16666666666666666,
 'nyeri': 0.3333333333333333,
 'pipi': 0.16666666666666666,
 'testis': 0.16666666666666666}

In [0]:
#menghitung suatu kata muncul di berapa dokumen
def kemunculankata():
    count = {}
    for gejala in TFdict:
        for word in gejala:
            if word in count:
                count[word] += 1
            else:
                count[word] = 1
    return count

count = kemunculankata()

In [0]:
#menghitung inverse document frequency 
#rumus IDF = log(jumlah doc  / jumlah doc dengan term itu di dalamnya) 
def IDF():
    IDFdict = {}
    for word in count:
        IDFdict[word] = math.log(len(cleandata) / count[word])
    return IDFdict

IDFdict = IDF()

In [0]:
#menghitung TF/IDF score dengan rumus TF* IDF
def hitungTFIDF(TFdict):
  TFIDF = {}
    
  for word in TFdict:
    TFIDF[word] = TFdict[word] * IDFdict[word]
  return TFIDF

tfidf = [hitungTFIDF(x) for x in TFdict]

In [12]:
#contoh tfidf score dari salah satu gejala
tfidf[1]

{'kaku': 0.4383197371649977,
 'kepala': 0.28782313662425574,
 'kesadaran': 0.5756462732485115,
 'kuduk': 0.48900287567851825,
 'muntah': 0.28782313662425574,
 'penurunan': 0.40235947810852507,
 'proyektil': 0.5756462732485115,
 'sakit': 0.20117973905426254}

In [0]:
#Membuat TFIDF Vector untuk feature representationnya 

#buat list key (word) dari dictionary 
worddict = sorted(count.keys())

#Buat vektor TF/IDF yaitu sepanjang jumlah term yang ada di word
def vektorTFIDF(text):
  vektortfidf = len(worddict) * [0.0]
     
  for i, word in enumerate(worddict):  
    if word in text:
      vektortfidf[i] = text[word]
  return vektortfidf

vektortfidf = [vektorTFIDF(x) for x in tfidf]

In [17]:
#contoh value vektortfidf
vektortfidf[1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4383197371649977,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.28782313662425574,
 0.0,
 0.0,
 0.0,
 0.5756462732485115,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


### Similarity Calculation

In [39]:
test.index

Int64Index([2, 4, 18, 19, 29, 36, 67, 76], dtype='int64')

In [0]:
#Hitung similarity dengan cosine similiarity v1.v2/||v1||*||v2||

In [0]:
def mag(vector):
  return math.sqrt(sum(i**2 for i in vector))

def cosinesim(v1,v2):
  return np.dot(v1,v2)/(mag(v1)*mag(v2))

In [36]:
cosinesim(vektortfidf[0],vektortfidf[1])

0.2104916647678051

In [61]:
if 3 not in test.index:
  print('ye')

ye


In [69]:
#menghitung similarity dan apabila similarity paling besar, maka diagnosis dari test = diagnosis dari gejala yang mempunyai similarity paling besar
for x in test.index:
  similarity = []
  for i in range(len(vektortfidf)):
    if (i !=x and i not in test.index):
      co = cosinesim(vektortfidf[i], vektortfidf[x])
      similarity.append(co)

  maxx = similarity.index(max(similarity))
  print(maxx)
  print(cleandata['diagnosis'][maxx])
  print(max(similarity))
  test['diagnosis'][x] = cleandata['diagnosis'][maxx]


88
malaria bentuk benigma
0.3743727545494964
3
parotitis
0.32704631833773096


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


16
faringitis
0.30015003737120644
16
faringitis
0.18455370433124957
25
appendicitis
0.39774630245182896
41
shigellosis
0.5890311721394355
17
laringitis
0.26434632285779897
77
herpes simpleks tipe 1
0.18917556164109411


In [70]:
#Hasil gejala yang sudah diprediksi diagnosisnya 
test

,gejala,diagnosis
2,mata lengket mata berair pandangan kabur kelop...,malaria bentuk benigma
4,gusi bengkak gusi kemerahan gusi berdarah,parotitis
18,batuk minggu sesak napas nyeri dada dahak berd...,faringitis
19,demam menggigil suhu tubuh meningkat batuk ber...,faringitis
29,demam muntah diare cair ampas biji lombok fese...,appendicitis
36,nyeri kolik daerah pinggang malaise mual kenci...,shigellosis
67,suhu tubuh batuk disertai sputum purulent berw...,laringitis
76,ruam gatal terdri macula makula lesi demam tuj...,herpes simpleks tipe 1
